In [1]:
import datetime

import polars as pl
from zipfile import ZipFile

In [2]:
coins = [
    'BTC',
    'ETH',
    'BNB',
    'SOL',
    'XRP',
    'DOGE',
    'ADA',
    'TON',
    # 'AVA',
    'DOT',
]

In [3]:
ticker_books = {}

for coin in coins:
    df = pl.read_csv(ZipFile(f'{coin}USDT-bookTicker-2024-03-15.zip').read(f'{coin}USDT-bookTicker-2024-03-15.csv'))
    df = df.with_columns(pl.from_epoch(pl.col("transaction_time"), time_unit="ms"))
    df = df.with_columns(pl.from_epoch(pl.col("event_time"), time_unit="ms"))
    df = df.with_columns((pl.col('best_bid_price') * pl.col('best_bid_qty')).alias("best_bid_total"))
    df = df.with_columns(((pl.col('best_bid_price') - pl.col('best_bid_price').min()) / (pl.col('best_bid_price').max() - pl.col('best_bid_price').min())).alias('best_bid_price_normal'))
    ticker_books[coin] = df.group_by('transaction_time').median()

In [4]:
for delta in [5, 10, 15]:
    print(f'delta: {delta}')
    for coin_1 in coins:
        for coin_2 in coins:
            if coin_2 == coin_1:
                continue
            df_1 = ticker_books[coin_1]
            df_2 = ticker_books[coin_2]
            df_1 = df_1.with_columns(shifted_transaction_time=pl.col("transaction_time") - datetime.timedelta(minutes=delta))
            df = df_2.join(df_1, left_on="transaction_time", right_on="shifted_transaction_time")[['best_bid_price', 'best_bid_price_right']]
            df = df.rename({
                'best_bid_price': coin_2,
                'best_bid_price_right': coin_1,
            })
            print(df.corr())

delta: 5
shape: (2, 2)
┌──────────┬──────────┐
│ ETH      ┆ BTC      │
│ ---      ┆ ---      │
│ f64      ┆ f64      │
╞══════════╪══════════╡
│ 1.0      ┆ 0.893953 │
│ 0.893953 ┆ 1.0      │
└──────────┴──────────┘
shape: (2, 2)
┌──────────┬──────────┐
│ BNB      ┆ BTC      │
│ ---      ┆ ---      │
│ f64      ┆ f64      │
╞══════════╪══════════╡
│ 1.0      ┆ 0.772596 │
│ 0.772596 ┆ 1.0      │
└──────────┴──────────┘
shape: (2, 2)
┌──────────┬──────────┐
│ SOL      ┆ BTC      │
│ ---      ┆ ---      │
│ f64      ┆ f64      │
╞══════════╪══════════╡
│ 1.0      ┆ 0.699577 │
│ 0.699577 ┆ 1.0      │
└──────────┴──────────┘
shape: (2, 2)
┌──────────┬──────────┐
│ XRP      ┆ BTC      │
│ ---      ┆ ---      │
│ f64      ┆ f64      │
╞══════════╪══════════╡
│ 1.0      ┆ 0.882093 │
│ 0.882093 ┆ 1.0      │
└──────────┴──────────┘
shape: (2, 2)
┌──────────┬──────────┐
│ DOGE     ┆ BTC      │
│ ---      ┆ ---      │
│ f64      ┆ f64      │
╞══════════╪══════════╡
│ 1.0      ┆ 0.821405 │
│ 0.82140